# Linear Classification

In this lab you will implement parts of a linear classification model using the regularized empirical risk minimization principle. By completing this lab and analysing the code, you gain deeper understanding of these type of models, and of gradient descent.


## Problem Setting

The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal (1) and abnormal (0). The training data contains 80 SPECT images from which 22 binary features have been extracted. The goal is to predict the label for an unseen test set of 187 tomography images.

In [41]:
import urllib
import pandas as pd
import numpy as np
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

testfile = urllib.request.URLopener()
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.train", "SPECT.train")
testfile.retrieve("http://archive.ics.uci.edu/ml/machine-learning-databases/spect/SPECT.test", "SPECT.test")

df_train = pd.read_csv('SPECT.train',header=None)
df_test = pd.read_csv('SPECT.test',header=None)

train = df_train.values
test = df_test.values

y_train = train[:,0]
X_train = train[:,1:]
y_test = test[:,0]
X_test = test[:,1:]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
w = np.random.randn(22)
print(w)
print(w.T)

[-2.3509872  -0.05713294 -0.93779756  0.77741797 -0.38695241  1.76093211
  0.3947773   1.11434433 -0.08836121  0.37166612 -2.18818974 -2.1206884
 -0.53060544 -1.0061862  -0.35551717  0.23426643  0.86950982 -0.1660445
 -0.47112301 -0.60312883 -1.02718706  1.17417996]
[-2.3509872  -0.05713294 -0.93779756  0.77741797 -0.38695241  1.76093211
  0.3947773   1.11434433 -0.08836121  0.37166612 -2.18818974 -2.1206884
 -0.53060544 -1.0061862  -0.35551717  0.23426643  0.86950982 -0.1660445
 -0.47112301 -0.60312883 -1.02718706  1.17417996]


### Exercise 1

Analyze the function learn_reg_ERM(X,y,lambda) which for a given $n\times m$ data matrix $\textbf{X}$ and binary class label $\textbf{y}$ learns and returns a linear model $\textbf{w}$.
The binary class label has to be transformed so that its range is $\left \{-1,1 \right \}$. 
The trade-off parameter between the empirical loss and the regularizer is given by $\lambda > 0$. 
To adapt the learning rate the Barzilai-Borwein method is used.

Try to understand each step of the learning algorithm and comment each line.


In [93]:
def learn_reg_ERM(X,y,lbda):
    max_iter = 200
    e  = 0.001
    alpha = 1.
    np.random.seed(5)
    w = np.random.randn(X.shape[1]);
    for k in np.arange(max_iter):
        h = np.dot(X,w) #compute output
        l,lg = loss(h, y) # calculate loss and loss gradient
        print ('loss: {}'.format(np.mean(l)))
        r,rg = reg(w, lbda) # calculate regularisation value
        g = np.dot(X.T,lg) + rg # gradient
        if (k > 0):
            alpha = alpha * (np.dot(g_old.T,g_old))/(np.dot((g_old - g).T,g_old)) 
        w = w - alpha * g # update weight
        if (np.linalg.norm(alpha * g) < e):
            break
        g_old = g
    return w

loss: 1.0490584038582944
loss: 5.238635546313249
loss: 3.1824805388585107
loss: 1.7885229173035568
loss: 2.563589821865231
loss: 2.67792906133404
loss: 1.462462666701748
loss: 102.16606489613007
loss: 14.11159259334483
loss: 37.81235875481328
loss: 16.626304920321296
loss: 181.9
loss: 12.280508703144251
loss: 28.787305041756564
loss: 8.087761504938573
loss: 166.82706189692522
loss: 15.769615986153571
loss: 29.24896133550312
loss: 8.628133991306118
loss: 135.3274726403648
loss: 12.05378027544898
loss: 16.63141339563265
loss: 2.3876823561522413
loss: 26.358408076692605
loss: 1.88089547391029
loss: 1.2905003781235487
loss: 1.2057564183490712
loss: 0.8718097975041823
loss: 0.8093685603002762
loss: 0.9983498913564244
loss: 1.5123676996405784
loss: 1.074706623702789
loss: 0.7383802160671893
loss: 0.6395755498805278
loss: 0.7228113435801344
loss: 0.6417948973723517
loss: 0.6138804476451705
loss: 0.6090388018149946
loss: 0.6136034926498863
loss: 0.7183913384357081
loss: 0.5936782463016861
loss

array([-36.63403679,   2.88819041,  -6.87292319,   3.00763361,
       -39.28334079,  -4.701392  ,   3.12707681,  15.65693761,
        -2.05208799, -21.81320159,  10.47777281,   0.59721601,
        20.83610241,   0.47777281,   5.1791648 ,  25.41805121,
        20.2388864 ,  15.1791648 , -12.05208799,  -4.46250559,
         8.06735521,  -4.22361919])

### Exercise 2

Fill in the code for the function loss(h,y) which computes the hinge loss and its gradient. 
This function takes a given vector $\textbf{y}$ with the true labels $\in \left \{-1,1\right \}$ and a vector $\textbf{h}$ with the function values of the linear model as inputs. The function returns a vector $\textbf{l}$ with the hinge loss $\max(0, 1 − y_{i} h_{i})$ and a vector $\textbf{g}$ with the gradients of the hinge loss w.r.t $\textbf{h}$. (Note: The partial derivative of the hinge loss with respect to $\textbf{h}$  is $g_{i} = −y $ if $l_{i} > 0$, else $g_{i} = 0$)

In [43]:
def loss(h, y):
    ##################
    #INSERT CODE HERE#
    ##################
    l = np.maximum(0,1-y*h)
    g = np.where(l > 0, -y, 0) # or -y * (l > 0)
    return l, g

In [13]:
a = np.array([1,2,3,4,5,6])
b = np.array([3,1,-5,1,7,3])
np.where(a > 3, -b,0)

array([ 0,  0,  0, -1, -7, -3])

### Exercise 3

Fill in the code for the function reg(w,lambda) which computes the $\mathcal{L}_2$-regularizer and the gradient of the regularizer function at point $\textbf{w}$. 


$$r = \frac{\lambda}{2} \textbf{w}^{T}\textbf{w}$$

$$g = \lambda \textbf{w}$$

In [44]:
def reg(w, lbda):
    ##################
    #INSERT CODE HERE#
    ##################
    r = (lbda/2) * np.dot(w.T,w) * 1.
    g = lbda * w
    return r, g

### Exercise 4

Fill in the code for the function predict(w,x) which predicts the class label $y$ for a data point $\textbf{x}$ or a matrix $X$ of data points (row-wise) for a previously trained linear model $\textbf{w}$. If there is only a data point given, the function is supposed to return a scalar value. If a matrix is given a vector of predictions is supposed to be returned.

In [64]:
def predict(w, X):
    ##################
    #INSERT CODE HERE#
    ##################
    preds = np.where(np.dot(X,w) <= 0, 0, 1)
    return preds

In [87]:
w = learn_reg_ERM(X_train,np.where(y_train == 0, -1, y_train),0.0001)
np.sum(predict(w, X_test)== y_test)/len(y_test)

loss: 1.764763226118641
loss: 8.438830068223563
loss: 5.106389903905269
loss: 3.8275622733374646
loss: 2.6511614597117035
loss: 2.154400875141779
loss: 1.1995298694131749
loss: 0.989722849226798
loss: 0.828967299455017
loss: 0.7995984535391586
loss: 0.7132537764414422
loss: 0.6835164020488783
loss: 775.7374999999431
loss: 1160.424462015542
loss: 354.1734901524022
loss: 98.0904551963769
loss: 297.979592316903
loss: 416.4455776001611
loss: 149.8751555770516
loss: 70.27844741855475
loss: 50.092218163472616
loss: 43.49420143560234
loss: 26.05197375419437
loss: 21.818386636004924
loss: 63.90881364116818
loss: 61.447760938923125
loss: 18.240400152261987
loss: 15.076991507023132
loss: 20.276557831716136
loss: 12.929565081914086
loss: 11.822391420366454
loss: 15.86241727121326
loss: 11.094230368392829
loss: 10.695375129384626
loss: 10.602348091049866
loss: 9.858559872059327
loss: 9.637135035776302
loss: 10.294402599863052
loss: 9.75387679107681
loss: 9.520951075089624
loss: 9.18970125100846
lo

0.6631016042780749

### Exercise 5

#### 5.1 
Train a linear model on the training data and classify all 187 test instances afterwards using the function predict. 
Please note that the given class labels are in the range $\left \{0,1 \right \}$, however the learning algorithm expects a label in the range of $\left \{-1,1 \right \}$. Then, compute the accuracy of your trained linear model on both the training and the test data. 

In [94]:
##################
#INSERT CODE HERE#
##################
y_train = np.where(y_train == 0, -1, y_train)
lbda = 0.0001
model = learn_reg_ERM(X_train,y_train,lbda)
preds = predict(model,X_test)
accuracy = (np.sum(preds == y_test)*100)/len(y_test)
print('Accuracy on test_data = ',accuracy)

loss: 1.4738741465975802
loss: 10.636001034611189
loss: 5.635684817740274
loss: 181250.650000003
loss: 24042.200572618945
loss: 60106.853442870524
loss: 23824.006906191375
loss: 181250.65
loss: 8827.493392650344
loss: 6283.224231197714
loss: 11675.963806485835
loss: 2413.4054427935644
loss: 1239.6463369488465
loss: 4823.637694653042
loss: 1600.6176065512923
loss: 652.146922998109
loss: 584.370887901542
loss: 476.0863186318549
loss: 380.8800379710348
loss: 387.73280479322455
loss: 399.27653955226253
loss: 320.83304187851604
loss: 294.65684938140976
loss: 252.24889254485387
loss: 283.6503403646302
loss: 367.7303128047353
loss: 231.20194392204561
loss: 207.781659445481
loss: 201.8633271442271
loss: 189.6908611747104
loss: 194.0224991142582
loss: 181.33675190171624
loss: 180.76947759114137
loss: 174.801111931447
loss: 170.85182050899874
loss: 171.624829682261
loss: 168.42288103740484
loss: 6875.712500000173
loss: 5689.898372628648
loss: 2874.752066487426
loss: 2933.8145038617986
loss: 1049

#### 5.2
Compare the accuracy of the linear model with the accuracy of a random forest and a decision tree on the training and test data set.

In [95]:
##################
#INSERT CODE HERE#
##################

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

clf = RandomForestClassifier()
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
accuracy = (np.sum(preds == y_test)*100)/len(y_test)
print('Accuracy on test_data using Random Forest = ',accuracy)

clf_tree = DecisionTreeClassifier()
clf_tree.fit(X_train, y_train)
preds = clf_tree.predict(X_test)
accuracy = (np.sum(preds == y_test)*100)/len(y_test)
print('Accuracy on test_data using Decision Tree = ',accuracy)

Accuracy on test_data using Random Forest =  68.98395721925134
Accuracy on test_data using Decision Tree =  60.42780748663102
